In [39]:
from pyspark.sql import SparkSession

In [40]:

spark = SparkSession.builder.appName("twitter_transform")\
    .master("local[*]")\
    .config("spark.driver.host", "127.0.0.1")\
    .getOrCreate()

In [41]:
path = "../datalake/twitter_spark"
df = spark.read.format("json").load(path)

In [42]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{159663454011354...|{[{2022-11-26T22:...|{1665871041879474...|  2023-06-05|
|[{163102162644700...|{[{2023-03-01T20:...|{1665870356694069...|  2023-06-05|
|[{328700883, 1665...|{[{2011-07-03T21:...|{1665869734179422...|  2023-06-05|
|[{161106447754107...|{[{2023-01-05T18:...|{1665869200181665...|  2023-06-05|
|[{165690748272235...|{[{2023-05-12T06:...|{1665868754691997...|  2023-06-05|
|[{141839209615164...|{[{2021-07-23T02:...|{1665868073113473...|  2023-06-05|
|[{163464544704946...|{[{2023-03-11T20:...|{1665867465568530...|  2023-06-05|
|[{161106447754107...|{[{2023-01-05T18:...|{1665866916005650...|  2023-06-05|
|[{328700883, 1665...|{[{2011-07-03T21:...|{1665866299447058...|  2023-06-05|
|[{134397719026262...|{[{2020-12-29T17:...|{1665865540772982...|